In [1]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

In [2]:
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import gc

In [3]:
%reload_ext autoreload
%autoreload 2

## KNN

In [4]:
class KNN:
    def __init__(self, k, dist_method='l2'):
        self.k = k
        self.dist_method=dist_method
        if dist_method not in ['l2', 'cosine']:
            raise ValueError("Please use either L2-distance or Cosine distance")
        # initialize training data to None in the Constructor
        self.x = None
        self.y = None
    
    def l2_norm(self, a):
        return np.sqrt(np.sum(np.square(a)))
    
    def calc_dist(self, a, b):
        if self.dist_method=='l2':
            return self.l2_norm(a-b)
        elif self.dist_method=='cosine':
            return np.dot(a,b)/(self.l2_norm(a) * self.l2_norm(b))
        
    def fit(self, x, y):
        self.x = x
        self.y = y
    
    def find_majority(self, labels):
        counts = Counter(labels)
        output = counts.most_common()[0][0]
        return output
    
    def predict(self, x_test):
        #find the k nearest neighbors
        preds = []
        for x_t in x_test:
            distances = [(self.calc_dist(x_train, x_t), y_train) for x_train, y_train in zip(self.x, self.y)]
            neighbor_labels = [y_tr for x_tr, y_tr in sorted(distances, key=lambda x:x[0])[:self.k]]
            preds.append(self.find_majority(neighbor_labels))
        
        return preds

In [5]:
data = load_iris()
X, y = data['data'], data['target']
SEED = 4213
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=SEED)
print(X_train.shape, X_test.shape)

(127, 4) (23, 4)


In [6]:
#define a hyperparameter
k_list = [3,5,7]

In [7]:
for k in k_list:
    knn_sklearn = KNeighborsClassifier(n_neighbors=k)
    knn_ = KNN(k=k)
    #fit
    knn_sklearn.fit(X_train, y_train)
    knn_.fit(X_train, y_train)
    #pred
    y_pred_sklearn = knn_sklearn.predict(X_test)
    y_pred_me = knn_.predict(X_test)
    print(f"k={k}, Sklearn implementation accu score: {accuracy_score(y_pred_sklearn, y_test)}\nMy implementation accu score: {accuracy_score(y_pred_me, y_test)}")

k=3, Sklearn implementation accu score: 1.0
My implementation accu score: 1.0
k=5, Sklearn implementation accu score: 1.0
My implementation accu score: 1.0
k=7, Sklearn implementation accu score: 1.0
My implementation accu score: 1.0


## KMeans

In [8]:
import random

In [9]:
x=np.array([1,2,3,4,1,1,2,1])
y = np.random.rand(8)
x, y

(array([1, 2, 3, 4, 1, 1, 2, 1]),
 array([0.01252186, 0.9180744 , 0.17612495, 0.17832535, 0.77903934,
        0.37883592, 0.28059209, 0.10873536]))

In [10]:
#[0,4,5,7]
y[x==1]

array([0.01252186, 0.77903934, 0.37883592, 0.10873536])

In [11]:
def _calc_l2(x):
    return np.sqrt(np.sum(np.square(x), axis=1))

x = np.random.rand(3)
y = np.random.rand(5,3)

_calc_l2(x-y)

array([0.44485395, 0.49865611, 0.95944225, 1.14000097, 0.87161578])

In [21]:
class KMeansV1:
    def __init__(self, n_clusters, method='l2'):
        self.n_clusters = n_clusters
        self.X = None
        if method not in ['l2','cosine']:
            raise ValueError("Please use either L2-distance or Cosine distance")
        self.method = method
    
    def _calc_l2(self, x):
        return np.sqrt(np.sum(np.square(x)))
    
    def _calc_dist(self, a, b):
        if self.method=='l2':
            return self._calc_l2(a-b)
        elif self.method=='cosine':
            return np.dot(a,b)/(self._calc_l2(a)*self._calc_l2(b))
    
    def fit(self, x):
        self.X = x
        N, d = x.shape
        #initialize cluster labels for each data point
        self.cluster_labels = np.zeros(N)
        #initialize cluster centers
        self.cluster_centers = [self.X[random.randint(0,N-1)] for _ in range(self.n_clusters)]
        #iteratively update cluster label for each data point
        stop = False
        while not stop:
            stop = True
            for index, x_tr in enumerate(x):
                #assign x_tr to the cluser which has the closest distance to the cluster center
                cur_cluster = np.argmin([self._calc_dist(x_tr, centroid) for centroid in self.cluster_centers])
                prev_cluster = self.cluster_labels[index]
                # does not converge
                if cur_cluster != prev_cluster:
                    self.cluster_labels[index] = cur_cluster
                    stop = False
            
            #update cluster centroids
            for i in range(self.n_clusters):
                self.cluster_centers[i] = np.mean(self.X[self.cluster_labels==i], axis=0)
                
    def predict(self, x_test):
        preds = []
        for x_te in x_test:
            cluster_label = np.argmin([self._calc_dist(x_te, centroid) for centroid in self.cluster_centers])
            preds.append(cluster_label)
        
        return preds        

In [28]:
for k in k_list:
    kmeans_sklearn = KMeans(n_clusters=k)
    kmeans_me = KMeansV1(n_clusters=k)
    
    kmeans_sklearn.fit(X_train)
    kmeans_me.fit(X_train)
    
    sklearn_pred = kmeans_sklearn.predict(X_test)
    my_pred = kmeans_me.predict(X_test)
    #cluster center labels can be different. As long as data are grouped into the same cluster, implementation is correct
    print(f"SKlearn prediction: {Counter(sklearn_pred)}\nMy prediction: {Counter(my_pred)}")

SKlearn prediction: Counter({1: 9, 0: 8, 2: 6})
My prediction: Counter({1: 9, 0: 8, 2: 6})
SKlearn prediction: Counter({2: 8, 4: 5, 1: 4, 3: 3, 0: 3})
My prediction: Counter({2: 8, 0: 6, 3: 3, 1: 3, 4: 3})
SKlearn prediction: Counter({6: 5, 3: 4, 0: 4, 4: 3, 2: 3, 1: 3, 5: 1})
My prediction: Counter({3: 5, 1: 4, 0: 4, 6: 3, 5: 3, 4: 3, 2: 1})


In [25]:
kmeans_sklearn.cluster_centers_

array([[7.38888889, 3.12222222, 6.18888889, 2.02222222],
       [4.69411765, 3.12352941, 1.41176471, 0.21764706],
       [5.96470588, 2.76470588, 4.98235294, 1.76470588],
       [6.54736842, 3.08947368, 5.50526316, 2.15789474],
       [5.46      , 2.565     , 3.855     , 1.175     ],
       [6.31      , 2.945     , 4.52      , 1.42      ],
       [5.236     , 3.672     , 1.512     , 0.268     ]])

In [26]:
kmeans_me.cluster_centers

[array([5.01666667, 3.45      , 1.47142857, 0.24761905]),
 array([6.54736842, 3.11578947, 5.46315789, 2.15263158]),
 array([7.15      , 2.9       , 5.98333333, 1.83333333]),
 array([6.20454545, 2.94545455, 4.46818182, 1.39545455]),
 array([7.575, 3.3  , 6.4  , 2.25 ]),
 array([5.9875 , 2.75   , 5.0125 , 1.78125]),
 array([5.43333333, 2.52222222, 3.81666667, 1.16666667])]